In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [3]:
df = pd.read_csv("data/dataset.csv")

In [4]:
print(df.head(1).product_url.to_string())

0    https://www.flipkart.com/pixie-fashion-solid-m...


In [5]:
def count_nulls(df):
    for column in df.columns:
        null_count = df[column].isnull().sum()
        print(f'The number of null values in the {column} column is: {null_count}')



In [6]:

df = df.dropna(subset=['category'])
df["brand"] = df["brand"].fillna("None")
count_nulls(df)

The number of null values in the product_id column is: 0
The number of null values in the Product_name column is: 0
The number of null values in the product_url column is: 0
The number of null values in the category column is: 0
The number of null values in the brand column is: 0
The number of null values in the price column is: 0
The number of null values in the discount column is: 0
The number of null values in the reviews column is: 0
The number of null values in the ratings column is: 0
The number of null values in the image_link column is: 0


In [7]:
!pip install transformers


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [10]:
def to_lower(df,columns):
    for column in columns:
        print(f'processing column {column}')
        df[column] = df[column].str.lower()
    return df
columns = ["Product_name","category"]

df = to_lower(df,columns)

processing column Product_name
processing column category


In [11]:
df.head()

,product_id,Product_name,product_url,category,brand,price,discount,reviews,ratings,image_link
0,TSHGFKAKNK8RUTDP,pack of 2 men's tshirt combo pack of 2 men sol...,https://www.flipkart.com/pixie-fashion-solid-m...,home>>clothing and accessories>>topwear>>t-shi...,PIXIE FASH,₹682,47% off,155 ratings and 13 reviews,3.8,https://rukminim2.flixcart.com/image/832/832/l...
1,TSHGDRHGWPJQ3DCB,men printed hooded neck cotton blend black t-s...,https://www.flipkart.com/denim-art-fashions-pr...,home>>clothing and accessories>>topwear>>t-shi...,DENIM ART FASHI,₹296,70% off,385 ratings and 16 reviews,3.8,https://rukminim2.flixcart.com/image/832/832/l...
2,SARGN39H4EP5WRVM,embroidered bollywood georgette saree (magenta),https://www.flipkart.com/anjavi-fashion-embroi...,home>>clothing and accessories>>sarees and sar...,ANJAVI FASH,₹294,88% off,"3,869 ratings and 204 reviews",3.6,https://rukminim2.flixcart.com/image/832/832/x...
3,SARFZP5YSGHHMQNG,floral print bollywood lycra blend saree (black),https://www.flipkart.com/ejoty-fashion-floral-...,home>>clothing and accessories>>sarees and sar...,Ejoty Fash,₹262,91% off,"26,476 ratings and 2,025 reviews",3.9,https://rukminim2.flixcart.com/image/832/832/x...
4,SARG7YSBQZTGBSGK,embroidered banarasi georgette saree (orange),https://www.flipkart.com/disha-fashion-embroid...,home>>clothing and accessories>>sarees and sar...,Disha Fash,₹599,62% off,"4,874 ratings and 289 reviews",3.8,https://rukminim2.flixcart.com/image/832/832/x...


In [12]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Product_name'] + " " + df['category'] + " " + df["brand"])

In [13]:
import json

In [14]:
from typing import List

def search_products(product_names: str) -> str:
    product_names = product_names.split(",")
    no_of_product_response=5
    """Meathod to get the products as per user query"""
    final_result=[]
    
    for product in product_names:
        print(f'searching for {product}')
        query_vec = tfidf_vectorizer.transform([product.lower()])
        cosine_similarities = linear_kernel(query_vec, tfidf_matrix).flatten()
        relevant_indices = cosine_similarities.argsort()[-no_of_product_response:][::-1]

        results = df.iloc[relevant_indices].copy()
        results['similarity_score'] = cosine_similarities[relevant_indices]
        results=results[["Product_name","product_url","image_link"]]
        results = results.to_dict(orient="records")
        final_result.append({product:results})
    
    return json.dumps(final_result)


In [51]:
from IPython.core.display import display, HTML

def display_results(results):
    results = json.loads(results)
    html = '<div style="display:flex;flex-direction:column">' 
    for result in results:
        for key in result:
            html+=f'<h3>{key}</h3>'
            for product in result[key]:
                
                html+=f'''
                <div style="display:flex;align-item:center; flex-direction:column;margin-top:10px;">
                <img src="{product["image_link"]}" width="100" />
                <a href={product["product_url"]}>{product["Product_name"]}</a>
                <div/>
                '''    
    html+="</div>"
    display(HTML(html))




/tmp/ipykernel_11619/3990681771.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [52]:
results = search_products("Blue shirt,jeans")
display_results(results)

searching for Blue shirt
searching for jeans


In [67]:
!pip install -q langchain

^C
ERROR: Operation cancelled by user


In [15]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
open_ai_api= user_secrets.get_secret("open_ai_api")
import os
os.environ["OPENAI_API_KEY"] =open_ai_api

In [16]:
!pip install -q openai

In [68]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import StructuredTool

tools = [
    StructuredTool.from_function(search_products,
        description='''useful when user wants to searh for any products. The input of this tool should be a  should be a comma separated list of product_names. For example, 'blue shirt,jeans' would be the input if you wanted to seach blue shirt and jeans together  '''
#         description='''useful when user wants to searh for any products. The input of this tool should be a  should be a comma separated list of product_names for example:'["shirt","jeans"]' , it returns json dump format of  ["shirt":[{product_name:"..",product_url:".."}]]'''
                                 
    )
]


In [69]:
from langchain.schema import SystemMessage

In [70]:
from langchain import  LLMChain
# prompt = ChatPromptTemplate.from_messages(
#     [
#         SystemMessage(
#             content=("You are kind and humble assistant at flipkart. You show users relevant products with the help of your tools")
#         ),
#         HumanMessagePromptTemplate.from_template("{text}"),
#     ]
# )
system_message = SystemMessage(content="You are kind and humble assistant at flipkart. You show users relevant products with the help of your tools.")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [71]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)


In [72]:
from langchain.agents import OpenAIFunctionsAgent

In [73]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

In [74]:
agent = OpenAIFunctionsAgent(
    tools=tools,
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [75]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,memory=memory)

In [76]:
agent_executor.run("Show me some blue shirt and red pants")



> Entering new AgentExecutor chain...

Invoking: `search_products` with `{'product_names': 'blue shirt,red pants'}`


searching for blue shirt
searching for red pants
[{"blue shirt": [{"Product_name": "embroidered bollywood chiffon saree\u00a0\u00a0(dark blue, light blue)", "product_url": "https://www.flipkart.com/abhilipsa-embroidered-bollywood-chiffon-saree/p/itm302f8dac09a0c?pid=SARGQHXVHFFR6HAZ&lid=LSTSARGQHXVHFFR6HAZXG3QDD&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_14_548&otracker=search&otracker1=search&fm=organic&iid=b4f9978d-0b88-42de-968e-efdfd89dc9e7.SARGQHXVHFFR6HAZ.SEARCH&ppt=sp&ppn=sp&ssid=ue15xtr6jk0000001692253360940&qH=ec025fba94606a13"}, {"Product_name": "men solid round neck polyester dark blue t-shirt", "product_url": "https://www.flipkart.com/kyk-solid-men-round-neck-dark-blue-t-shirt/p/itm5478a9836e19b?pid=TSHGJPVHG2SEFJDW&lid=LSTTSHGJPVHG2SEFJDW0OOZAK&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_2_42&otracker=search&otracker1=search&fm=

'Here are some blue shirts and red pants that you might like:\n\nBlue Shirts:\n1. Embroidered Bollywood Chiffon Saree (Dark Blue, Light Blue) - [Link](https://www.flipkart.com/abhilipsa-embroidered-bollywood-chiffon-saree/p/itm302f8dac09a0c?pid=SARGQHXVHFFR6HAZ&lid=LSTSARGQHXVHFFR6HAZXG3QDD&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_14_548&otracker=search&otracker1=search&fm=organic&iid=b4f9978d-0b88-42de-968e-efdfd89dc9e7.SARGQHXVHFFR6HAZ.SEARCH&ppt=sp&ppn=sp&ssid=ue15xtr6jk0000001692253360940&qH=ec025fba94606a13)\n2. Men Solid Round Neck Polyester Dark Blue T-Shirt - [Link](https://www.flipkart.com/kyk-solid-men-round-neck-dark-blue-t-shirt/p/itm5478a9836e19b?pid=TSHGJPVHG2SEFJDW&lid=LSTTSHGJPVHG2SEFJDW0OOZAK&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_2_42&otracker=search&otracker1=search&fm=organic&iid=en_YD9As_sLgF5ovsK4Cm59Vp0fYwQNW7_7Ay4JyDbLsqDZgip-4L0c62mBpwWXqpME7Up3UYqlpHv0uEbYaAiqJA%3D%3D&ppt=sp&ppn=sp&ssid=wtgfsqzvi8000000169225335887&qH=ec025fba

In [77]:
agent_executor.run("Suggest me a dress to wear at my birthday. I am 20 year old boy in am engineering college")



> Entering new AgentExecutor chain...

Invoking: `search_products` with `{'product_names': "men's party wear"}`


searching for men's party wear
[{"men's party wear": [{"Product_name": "girls party(festive) top skirt\u00a0\u00a0(blue)", "product_url": "https://www.flipkart.com/fashion-dream-girls-party-festive-top-skirt/p/itmba1ed5af3b738?pid=KPBGDY9BSSD6JYRF&lid=LSTKPBGDY9BSSD6JYRFCPTB5I&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_2_57&otracker=search&otracker1=search&fm=organic&iid=en_YD9As_sLgF5ovsK4Cm59Vp0fYwQNW7_7Ay4JyDbLsqAIPW87pXhugAK2HS0w-2aHzPCF7zrp3eX_QvaOI5AKRQ%3D%3D&ppt=sp&ppn=sp&ssid=wtgfsqzvi80000001692253284587&qH=ec025fba94606a13"}, {"Product_name": "girls calf length party dress\u00a0\u00a0(blue, fashion sleeve)", "product_url": "https://www.flipkart.com/sanskruti-girls-calf-length-party-dress/p/itmf76caa5693385?pid=KDEGRZQDVKUUGJTZ&lid=LSTKDEGRZQDVKUUGJTZ4SBEBP&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_1_35&otracker=search&otracker1=searc

'Here are some suggestions for party wear dresses for your birthday:\n\n1. Girls Party (Festive) Top Skirt (Blue) - [Link](https://www.flipkart.com/fashion-dream-girls-party-festive-top-skirt/p/itmba1ed5af3b738?pid=KPBGDY9BSSD6JYRF&lid=LSTKPBGDY9BSSD6JYRFCPTB5I&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_2_57&otracker=search&otracker1=search&fm=organic&iid=en_YD9As_sLgF5ovsK4Cm59Vp0fYwQNW7_7Ay4JyDbLsqAIPW87pXhugAK2HS0w-2aHzPCF7zrp3eX_QvaOI5AKRQ%3D%3D&ppt=sp&ppn=sp&ssid=wtgfsqzvi80000001692253284587&qH=ec025fba94606a13)\n\n2. Girls Calf Length Party Dress (Blue, Fashion Sleeve) - [Link](https://www.flipkart.com/sanskruti-girls-calf-length-party-dress/p/itmf76caa5693385?pid=KDEGRZQDVKUUGJTZ&lid=LSTKDEGRZQDVKUUGJTZ4SBEBP&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_1_35&otracker=search&otracker1=search&iid=5c09f0a6-04c5-495d-b1f1-bda211d8600d.KDEGRZQDVKUUGJTZ.SEARCH&ssid=wr8517hysw0000001692253277472&qH=ec025fba94606a13)\n\n3. Girls Calf Length Party Dress (Red, 3

In [78]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Show me some blue shirt and red pants', additional_kwargs={}, example=False),
  AIMessage(content='Here are some blue shirts and red pants that you might like:\n\nBlue Shirts:\n1. Embroidered Bollywood Chiffon Saree (Dark Blue, Light Blue) - [Link](https://www.flipkart.com/abhilipsa-embroidered-bollywood-chiffon-saree/p/itm302f8dac09a0c?pid=SARGQHXVHFFR6HAZ&lid=LSTSARGQHXVHFFR6HAZXG3QDD&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_14_548&otracker=search&otracker1=search&fm=organic&iid=b4f9978d-0b88-42de-968e-efdfd89dc9e7.SARGQHXVHFFR6HAZ.SEARCH&ppt=sp&ppn=sp&ssid=ue15xtr6jk0000001692253360940&qH=ec025fba94606a13)\n2. Men Solid Round Neck Polyester Dark Blue T-Shirt - [Link](https://www.flipkart.com/kyk-solid-men-round-neck-dark-blue-t-shirt/p/itm5478a9836e19b?pid=TSHGJPVHG2SEFJDW&lid=LSTTSHGJPVHG2SEFJDW0OOZAK&marketplace=FLIPKART&q=trending+fashion&store=clo&srno=s_2_42&otracker=search&otracker1=search&fm=organic&iid=en_YD9As_sLgF5ovsK4Cm5